# Installing Theano
`` pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git``

# Installing Tensorflow
``pip install tensorflow``

# Installing Keras
``pip install --upgrade keras``

In [1]:
# Part 1 - Data Preprocessing
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Encoding categorical data
#print(X[:,1])
print(X[0,0],X[0,2])
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
#print(X[:,1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]
#print(X[:,1])
print(X[0,2])

619 Female
619.0


In [4]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
print(X_train[0])

[  0.00000000e+00   1.00000000e+00   6.67000000e+02   0.00000000e+00
   3.40000000e+01   5.00000000e+00   0.00000000e+00   2.00000000e+00
   1.00000000e+00   0.00000000e+00   1.63830640e+05]


In [5]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [6]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [7]:
# Initialising the ANN
classifier = Sequential()

In [8]:
# Adding the input layer and the first hidden layer
classifier.add(Dense(units= 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

In [9]:
# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

In [10]:
# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

In [13]:
# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#'categorical_crossentropy' for classification with multiple classes

In [14]:
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 1s - loss: 0.4888 - acc: 0.7959     
Epoch 2/100
8000/8000 [==============================] - 1s - loss: 0.4222 - acc: 0.8017     
Epoch 3/100
8000/8000 [==============================] - 1s - loss: 0.4144 - acc: 0.8282     
Epoch 4/100
8000/8000 [==============================] - 1s - loss: 0.4096 - acc: 0.8334     
Epoch 5/100
8000/8000 [==============================] - 1s - loss: 0.4061 - acc: 0.8345     
Epoch 6/100
8000/8000 [==============================] - 1s - loss: 0.4041 - acc: 0.8340     
Epoch 7/100
8000/8000 [==============================] - 1s - loss: 0.4019 - acc: 0.8352     
Epoch 8/100
8000/8000 [==============================] - 1s - loss: 0.4009 - acc: 0.8346     
Epoch 9/100
8000/8000 [==============================] - 1s - loss: 0.4005 - acc: 0.8357     
Epoch 10/100
8000/8000 [==============================] - 1s - loss: 0.3993 - acc: 0.8330     
Epoch 11/100
8000/8000 [==============================] - 1

8000/8000 [==============================] - 1s - loss: 0.3440 - acc: 0.8631     
Epoch 87/100
8000/8000 [==============================] - 1s - loss: 0.3441 - acc: 0.8625     
Epoch 88/100
8000/8000 [==============================] - 1s - loss: 0.3429 - acc: 0.8606     
Epoch 89/100
8000/8000 [==============================] - 1s - loss: 0.3436 - acc: 0.8614     
Epoch 90/100
8000/8000 [==============================] - 1s - loss: 0.3430 - acc: 0.8617     
Epoch 91/100
8000/8000 [==============================] - 1s - loss: 0.3435 - acc: 0.8601     
Epoch 92/100
8000/8000 [==============================] - 1s - loss: 0.3435 - acc: 0.8602     
Epoch 93/100
8000/8000 [==============================] - 1s - loss: 0.3434 - acc: 0.8594     
Epoch 94/100
8000/8000 [==============================] - 1s - loss: 0.3430 - acc: 0.8591     
Epoch 95/100
8000/8000 [==============================] - 1s - loss: 0.3435 - acc: 0.8604     
Epoch 96/100
8000/8000 [==============================] - 1s - 

In [25]:
# Part 3 - Making predictions and evaluating the model

# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)
print(y_pred[:100])
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [ True]
 [ True]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]]
[[1525   70]
 [ 208  197]]


In [27]:
#calculate accuracy
acc=(1500+197)/2000
print(acc)

0.8485
